In [1]:
"""This creates and trains a model given some random data. It can be
used to test your installation of the relevant libraries.
"""

import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
# from tensorflow.keras import callbacks as cb

tfkl = tf.keras.layers
tfpl = tfp.layers
tfd = tfp.distributions
print("Loaded the libraries.")
print(tf.config.list_physical_devices())
MODEL_DIR = "./test_model/"

Loaded the libraries.
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


In [2]:
# Generate random data

data = np.load("data/training_data.npy")
print(data.shape)

N = data.shape[0]
print(f"{N = }")

X = data[:, 0:2, :].reshape(N, 4)
Y = data[:, 2:4, :].reshape(N, 4)

Y_diff = data[:, 2:4, :] - data[:, 0:2, :] 

print(X.shape)
print(Y.shape)

(31109, 4, 2)
N = 31109
(31109, 4)
(31109, 4)


In [3]:
# Make simple model

mirrored_strategy = tf.distribute.MirroredStrategy()
with mirrored_strategy.scope():
    
    # 15 here comes from the number of alphas, the number of mu's and the entries of the covariance matrix.
    
    model = tf.keras.Sequential(
         [tfkl.Dense(32, activation='tanh'),
         tfkl.Dense(2 * 15, activation=None),
         tfpl.MixtureSameFamily(2, tfpl.MultivariateNormalTriL(4))])

print("Built the model.")


# Train model

def nll(data_point, tf_distribution):
    """Negative log likelihood."""
    return -tf_distribution.log_prob(data_point)


LOSS = nll
BATCH_SIZE = 32
LEARNING_RATE = 1e-3
EPOCHS = 3
OPTIMISER = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
VALIDATION_SPLIT = 0.2

model.compile(loss=LOSS, optimizer=OPTIMISER)

History = model.fit(
    X,
    Y,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    shuffle=True,
    validation_split=VALIDATION_SPLIT,
    verbose=2,
)
print("Trained the model.")

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


Built the model.
Epoch 1/3


2024-03-24 09:32:02.293713: W tensorflow/core/framework/dataset.cc:959] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


778/778 - 2s - loss: -3.6014e-02 - val_loss: -1.6660e-01 - 2s/epoch - 2ms/step
Epoch 2/3
778/778 - 1s - loss: -9.0126e-01 - val_loss: -1.8200e-01 - 531ms/epoch - 683us/step
Epoch 3/3
778/778 - 1s - loss: -1.0233e+00 - val_loss: -3.2851e-01 - 524ms/epoch - 674us/step
Trained the model.
